# Tested solutions

In [1]:
# Hide warnings if there are any
# import warnings
# warnings.filterwarnings('ignore')

1) Load VGG model <- work well

In [ ]:
import os
import utils
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image

# Load an image.
image = utils.load_image(
    os.path.join("../dataset/test/01_TUMOR", "1D8F_CRC-Prim-HE-02_007b.jpg_Row_151_Col_1.jpg"), 224)

# Code snippet.
plt.imshow(image/255)
plt.axis('off')
plt.savefig("readme_example_input.png")

import keras.applications.vgg16 as vgg16
model, preprocess = vgg16.VGG16(), vgg16.preprocess_input

#####################
# Strip softmax layer
import innvestigate
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
x = preprocess(image[None])

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')

Using TensorFlow backend.








[<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>];[<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>];vgg16


2) Run with ARA model

In [ ]:
import os
import utils
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image

# Load an image.
image = utils.load_image(
    os.path.join("../dataset/test/01_TUMOR", "1D8F_CRC-Prim-HE-02_007b.jpg_Row_151_Col_1.jpg"), 224)

# Code snippet.
plt.imshow(image/255)
plt.axis('off')
plt.savefig("readme_example_input.png")

# Get model
import keras
print(keras.__version__)

from keras.models import model_from_json
with open('../model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('../ara_cnn.h5')
# model = keras.Model(inputs=model.input, outputs=[model.get_layer(name='main_output').output, model.get_layer(name='aux_output').output])
print("Loaded model from disk")

#####################
# Strip softmax layer
import innvestigate
"""Creates a new model w/o the final softmax activation."""
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
# x = preprocess(image[None])
x = innvestigate.utils.preprocess_images(image[None])
# x = image[None]

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')

3) Different way of loading model

In [ ]:
# layer.get_config()['activation']

import json
with open('../model.json', "r") as f:
    model_json = json.load(f)
    
# with open('../model.json', 'r') as f:
#     model_json = model_from_json(f.read())

from model_utils import single_class_accuracy
model = model_from_json(model_json, custom_objects={'fn': single_class_accuracy(0)},)
model.load_weights('../ara_cnn.h5')


# load weights into new model
# model = load_model('../ara_cnn.h5', custom_objects={'fn': single_class_accuracy(0)})
print("Loaded model from disk")

#####################
# Strip softmax layer
import innvestigate
"""Creates a new model w/o the final softmax activation."""
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
x = preprocess(image[None])

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')

4) Compile model (configures the model for training)

In [ ]:
with open('../model.json', 'r') as f:
    model = model_from_json(f.read())

# load weights into new model
model.load_weights('../ara_cnn.h5')
print("Loaded model from disk")

from keras.optimizers import Adam
from model_utils import single_class_accuracy
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", 
              loss_weights={"main_output": 0.9, "aux_output": 0.1}, metrics=["acc", single_class_accuracy(0)])

#####################
# Strip softmax layer
import innvestigate
"""Creates a new model w/o the final softmax activation."""
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
x = preprocess(image[None])

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')

5) Create new model

In [ ]:
with open('../model.json', 'r') as f:
    model = model_from_json(f.read())

# load weights into new model
model.load_weights('../ara_cnn.h5')

from keras.models import Model
from innvestigate.utils.keras.graph import pre_softmax_tensors
model2 = Model(inputs=model.inputs,
                              outputs=pre_softmax_tensors(model.outputs),
                              name=model.name)

#####################
# Strip softmax layer
import innvestigate
"""Creates a new model w/o the final softmax activation."""
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
x = preprocess(image[None])

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')

6) Remove layer

In [ ]:
import os
import utils
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image

# Load an image.
image = utils.load_image(
    os.path.join("../dataset/test/01_TUMOR", "1D8F_CRC-Prim-HE-02_007b.jpg_Row_151_Col_1.jpg"), 224)

# Code snippet.
plt.imshow(image/255)
plt.axis('off')
plt.savefig("readme_example_input.png")

# Get model
import keras
print(keras.__version__)

from keras.models import model_from_json
with open('../model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('../ara_cnn.h5')
print("Loaded model from disk")
for x in range(0, 8):
    model.layers.pop() 
print(model.summary())

#####################
# Strip softmax layer
import innvestigate
"""Creates a new model w/o the final softmax activation."""
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Add batch axis and preprocess
# x = preprocess(image[None])
x = innvestigate.utils.preprocess_images(image[None])
# x = image[None]

# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(x)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))

# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))
plt.axis('off')